In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [10]:
import zipfile
import os

# Unzip train.zip
with zipfile.ZipFile('MP_Data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

ValueError: Mountpoint must be in a directory that exists

In [6]:
!mv  MP_Data/content/drive/MyDrive/


mv: missing destination file operand after 'MP_Data/content/drive/MyDrive/'
Try 'mv --help' for more information.


In [4]:
data_path = '/content/drive/MyDrive/MP_Data'


In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [13]:
actions = np.array(['hello', 'thanks', 'iloveyou'])
label_map = {label:num for num, label in enumerate(actions)}
label_map
{'hello': 0, 'thanks': 1, 'iloveyou': 2}


{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [14]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')
# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [24]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
         try:
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
         except:
            pass
        if len(window) == sequence_length: # Only append if the window has the correct number of frames
            sequences.append(window)
            labels.append(label_map[action])



In [25]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [28]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [29]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [30]:
model.fit(X_train, y_train, epochs=2000)

Epoch 1/2000
3/3 [==============================] - 4s 248ms/step - loss: 1.7083 - categorical_accuracy: 0.3059
Epoch 2/2000
3/3 [==============================] - 1s 201ms/step - loss: 3.2718 - categorical_accuracy: 0.1882
Epoch 3/2000
3/3 [==============================] - 1s 202ms/step - loss: 6.9687 - categorical_accuracy: 0.2588
Epoch 4/2000
3/3 [==============================] - 1s 198ms/step - loss: 11.1434 - categorical_accuracy: 0.2941
Epoch 5/2000
3/3 [==============================] - 1s 160ms/step - loss: 10.8847 - categorical_accuracy: 0.2824
Epoch 6/2000
3/3 [==============================] - 1s 200ms/step - loss: 6.6963 - categorical_accuracy: 0.2941
Epoch 7/2000
3/3 [==============================] - 1s 205ms/step - loss: 2.7193 - categorical_accuracy: 0.3529
Epoch 8/2000
3/3 [==============================] - 1s 156ms/step - loss: 3.7846 - categorical_accuracy: 0.3882
Epoch 9/2000
3/3 [==============================] - 1s 217ms/step - loss: 2.0511 - categorical_accurac

In [34]:
res = model.predict(X_test)
actions[np.argmax(res[0])]

1/1 [==============================] - 0s 36ms/step


'thanks'

In [38]:
actions[np.argmax(y_test[0])]

'thanks'